In [141]:
def read_file(file_path):
    with open(file_path,'r',encoding='utf-8') as f:
        lines = f.readlines()
        
    row = lines[0]
    row_name = row.strip().split('\t')[1:]
    
    column_name = []
    data = []
    for line in lines[1:]:
        row_data = line.strip().split('\t')
        column_name.append(row_data[0])
        data.append([float(i) for i in row_data[1:]])
    return row_name,column_name,data
        
key_words,blog_titles,data = read_file('blogdata.txt')


# key_words.index('yahoo') # 3
# blog_titles.index('Quick Online Tips') # 85
# data[85][3]

In [139]:
# 构建的新的聚类结构
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0,id=None):
        self.left = left # 指向左子类
        self.right = right # 指向右子类
        self.vec = vec # 上文的data,即 词的频率
        self.id = id # 文档的ID
        self.distance = distance # 左子类和右子类的相似度

In [140]:
from math import sqrt
def pearson(v1,v2):
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    sum1sq = sum([pow(v,2) for v in v1])
    sum2sq = sum([pow(v,2) for v in v2])
    
    multi_sum = sum([v1[i] * v2[i] for i in range(len(v1))])
    
    num = multi_sum - (sum1 * sum2 / len(v1))
    den = sqrt((sum1sq - pow(sum1, 2) / len(v1)) * (sum2sq - pow(sum2,2) / len(v2)))
    if den == 0:
        return 0
    return 1.0 - num/den

In [148]:
'''
聚类函数 核心
算法核心是将文档两两pearson值计算，将两个距离最小的合并成一个新类，同时利用树的结构保存原来的关系，直至只有一个大类，算法结束
'''
def hcluster(rows,distance=pearson):
    # 初始时每个数据都是一类，用列表存储
    clust = [bicluster(rows[i],id=i) for i in range(len(rows))]
    
    distance_pair = {}
    currentclustid=-1 # -1 表示构建生成的类ID
    # 得到树根（即一个类）时退出循环
    while len(clust) > 1:
        lowestpair = (0,1)
        closest = distance(clust[0].vec, clust[1].vec)
        # 两两对比，寻找最小距离
        for i in range(len(clust)):
            for j in range(i+1, len(clust)):
                if (clust[i].id, clust[j].id) not in distance_pair:
                    distance_pair[(clust[i].id, clust[j].id)] = distance(clust[i].vec, clust[j].vec)
                
                d = distance_pair[(clust[i].id, clust[j].id)]
                if d < closest:
                    closest = d
                    lowestpair = (i,j)
        
        # 计算两个最小距离对应的类的平均值
        mergevec=[(clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]
        # 将上面两个类合并生成新的类，左右分支分别指向原来的类
        newcluster=bicluster(mergevec,left=clust[lowestpair[0]],right=clust[lowestpair[1]],distance=closest,id=currentclustid)
        
        currentclustid-=1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)

    return clust[0],distance_pair

result,result1 = hcluster(data)

In [146]:
# 字符版打印聚类，理解成树
def printclust(clust,labels=None,n=0):
    for i in range(n): 
        print(' ',end="")
    if clust.id<0:
        print('-')
    else:
        if labels==None:
            print(clust.id)
        else:
            print(labels[clust.id])

    if clust.left!=None:
        printclust(clust.left,labels=labels,n=n+1)
    if clust.right!=None:
        printclust(clust.right,labels=labels,n=n+1)
        
printclust(result,blog_titles)

-
 gapingvoid: "cartoons drawn on the back of business cards"
 -
  -
   Schneier on Security
   Instapundit.com
  -
   The Blotter
   -
    -
     MetaFilter
     -
      SpikedHumor
      -
       Captain's Quarters
       -
        Michelle Malkin
        -
         -
          NewsBusters.org - Exposing Liberal Media Bias
          -
           -
            Hot Air
            Crooks and Liars
           -
            Power Line
            Think Progress
         -
          Andrew Sullivan | The Daily Dish
          -
           Little Green Footballs
           -
            Eschaton
            -
             Talking Points Memo: by Joshua Micah Marshall
             Daily Kos
    -
     43 Folders
     -
      TechEBlog
      -
       -
        Mashable!
        Signum sine tinnitu--by Guy Kawasaki
       -
        -
         -
          Slashdot
          -
           MAKE Magazine
           Boing Boing
         -
          -
           Oilman
           -
            Online

In [149]:
from PIL import Image,ImageDraw
